In [1]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.15.0
ipykernel        : 6.25.0
ipywidgets       : 8.0.4
jupyter_client   : 7.4.9
jupyter_core     : 5.3.0
jupyter_server   : 1.23.4
jupyterlab       : 3.6.3
nbclient         : 0.5.13
nbconvert        : 6.5.4
nbformat         : 5.9.2
notebook         : 6.5.4
qtconsole        : 5.4.2
traitlets        : 5.7.1


# Introduction

### What is Yu-Gi-Oh

### How dataset was obtained

### Motivation for data analysis, Problem Statement

Questions We Are Looking To Answer 
- Predict price of card (Linear Regressions) \[attk/level/defence/set\] 
- Predict level based on attack and defence 
- Predict "effect monster" Type \[categorical\] based (Classification Tree) 

Pointers <br>
Make a point on how data is cleaned 
Still do EDA, and see the exceptions 
AKA showcase you know your stuff 
Can split monsters based on can summon normally, or need sacrifice to find. 
 - Can become categorical (bucketing, similar to histograms) 
Show the trend of powercreep over the sets

In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [3]:
data_file_location = 'card_data.csv'

yugioh_data = pd.read_csv(data_file_location,index_col=0)
yugioh_data.head(10) # quick look of data

,id,name,type,frameType,desc,race,archetype,card_sets,card_images,card_prices,atk,def,level,attribute,pend_desc,monster_desc,scale,linkval,linkmarkers,banlist_info
0,34541863,"""A"" Cell Breeding Device",Spell Card,spell,"During each of your Standby Phases, put 1 A-Co...",Continuous,Alien,"[{'set_name': 'Force of the Breaker', 'set_cod...","[{'id': 34541863, 'image_url': 'https://images...","[{'cardmarket_price': '0.10', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64163367,"""A"" Cell Incubator",Spell Card,spell,Each time an A-Counter(s) is removed from play...,Continuous,Alien,"[{'set_name': ""Gladiator's Assault"", 'set_code...","[{'id': 64163367, 'image_url': 'https://images...","[{'cardmarket_price': '0.07', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91231901,"""A"" Cell Recombination Device",Spell Card,spell,Target 1 face-up monster on the field; send 1 ...,Quick-Play,Alien,"[{'set_name': 'Invasion: Vengeance', 'set_code...","[{'id': 91231901, 'image_url': 'https://images...","[{'cardmarket_price': '0.10', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73262676,"""A"" Cell Scatter Burst",Spell Card,spell,"Select 1 face-up ""Alien"" monster you control. ...",Quick-Play,Alien,"[{'set_name': 'Strike of Neos', 'set_code': 'S...","[{'id': 73262676, 'image_url': 'https://images...","[{'cardmarket_price': '0.12', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,98319530,"""Infernoble Arms - Almace""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': 'Duelist Nexus', 'set_code': 'DU...","[{'id': 98319530, 'image_url': 'https://images...","[{'cardmarket_price': '0.00', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,37478723,"""Infernoble Arms - Durendal""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': '2021 Tin of Ancient Battles', '...","[{'id': 37478723, 'image_url': 'https://images...","[{'cardmarket_price': '0.16', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,64867422,"""Infernoble Arms - Hauteclere""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': '2021 Tin of Ancient Battles', '...","[{'id': 64867422, 'image_url': 'https://images...","[{'cardmarket_price': '0.02', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,90861137,"""Infernoble Arms - Joyeuse""",Spell Card,spell,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': '2021 Tin of Ancient Battles', '...","[{'id': 90861137, 'image_url': 'https://images...","[{'cardmarket_price': '0.18', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,80845034,"""The Sinful Spoils Hunter Fiend""",Spell Card,spell,"Add 1 ""Diabellestarr"" monster from your Deck o...",Quick-Play,Sinful Spoils,NaN,"[{'id': 80845034, 'image_url': 'https://images...","[{'cardmarket_price': '0.00', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44256816,1st Movement Solo,Spell Card,spell,If you control no monsters: Special Summon 1 L...,Normal,Melodious,"[{'set_name': '2015 Mega-Tin Mega Pack', 'set_...","[{'id': 44256816, 'image_url': 'https://images...","[{'cardmarket_price': '2.85', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
### Drop irrelevant columns about descriptions
yugioh_data = pd.read_csv(data_file_location,index_col=0)
yugioh_data = yugioh_data.drop(['desc', 'pend_desc', 'monster_desc'], axis=1)

### Extract price data into 5 columns from original column
import json
# Strip the starting and ending '[]' from column
yugioh_data['card_prices'] = yugioh_data['card_prices'].str.strip('[]')

# Convert the string in the 'card_prices' column to a dictionary
yugioh_data['card_prices'] = yugioh_data['card_prices'].apply(lambda x: json.loads(x.replace("'", '"')))

# Normalize the 'card_prices' column to create separate columns for each key in the dictionary
yugioh_data = pd.concat([yugioh_data, pd.json_normalize(yugioh_data['card_prices'])], axis=1)

# Prices are currently of type string, convert to float64
price_columns = ['cardmarket_price', 'tcgplayer_price', 'ebay_price', 'amazon_price', 'coolstuffinc_price']
#s = pd.Series(yugioh_data[price_columns])
yugioh_data[['cardmarket_price', 'tcgplayer_price', 'ebay_price', 'amazon_price', 'coolstuffinc_price']] = pd.to_numeric(yugioh_data[['cardmarket_price', 'tcgplayer_price', 'ebay_price', 'amazon_price', 'coolstuffinc_price']])

#pd.to_numeric(s)
#yugioh_data[price_columns] = pd.to_numeric(yugioh_data[price_columns])

# Now the DataFrame 'df' contains 5 new columns with the prices from the 'card_prices' column
yugioh_data.head()
yugioh_data.info()

### Find the mean price, ignoring null values
yugioh_data['avg_price'] = yugioh_data[price_columns].mean(axis=1)
yugioh_data.head()


TypeError: arg must be a list, tuple, 1-d array, or Series

#### Separate Level into ranges and bins 

In [ ]:
# Define the bins for the level ranges
bins = [1.0, 4.0, 6.0, float('inf')]

# Define the labels for the level ranges
labels = ['0', '1', '2']

# Create a new column 'level_group' to store the bucketed groups
yugioh_data['tribute'] = pd.cut(yugioh_data['level'], bins=bins, labels=labels, right=False)

# Now the DataFrame 'df' contains a new column 'level_group' with the cards bucketed into the specified level ranges
yugioh_data.head()

#### 

# Exploratory Data Analysis [EDA]

In [ ]:
# Information about the Variables
yugioh_data.info()

<b> Listing down observations of dataset: </b>

- 20 columns and 12815 rows.
- 1 variables identified as int64 by default, and it seems is indeed Numeric.
- 5 variables identified as float64 by default, and it seems are indeed Numeric.
- 14 variables identified as object by default, 
    - 4 variables are Strings, which represent card text and should not be used for Categorical values.
        - (name, desc, pend_desc, monster_desc)
    - 1 variables is a JSON object, which represent link data to card images and should not be used for Categorical values.
        - (card_images)
    - 3 variables are JSON objects, which represent Categorical values.
        - (card_sets, linkmarkers, banlist_info)
    - 5 variables are Strings, which represent Categorical values.
        - (type, frameType, race, archetype, attribute)
    - 1 variables is a JSON object, which represent multiple Numeric values.
        - (card_prices)


<b> Cleaning actions to be taken </b> <br>
    - Extract out values from relevant JSON objects (card_prices)
    
None of the variables/features seem to have any missing value (have to check again, carefully).
- card_prices is 


### Data Cleaning


In [ ]:
# Drop card text about descriptions
#yugioh_data = yugioh_data.drop(['desc', 'pend_desc', 'monster_desc'], axis=1)
#yugioh_data.head()

In [ ]:
# Visulising unique types for categorical values
unique_types = yugioh_data['type'].unique()
print(unique_types)

In [ ]:
# Remove non-monsters from dataset.
yugioh_data = yugioh_data[~yugioh_data['type'].isin(['Skill Card', 'Spell Card', 'Trap Card', 'Token'])]
yugioh_data.head()

# EDA OF NUMERIC VALUES


In [ ]:
# Extract numeric data variables
#numericalColumns = ["id", "atk", "def", "level","scale","linkval"]
numericalColumns = ["atk", "def", "level","scale","linkval"]
yugioh_data = pd.DataFrame(yugioh_data[numericalColumns])

# Summary Statistics for all Variables
yugioh_data.describe().round(2)

In [ ]:
# Draw the distributions of all variables len(numericalColumns)
f, axes = plt.subplots(5, 3, figsize=(18, 24))

yugioh_data = yugioh_data.reset_index(drop=True)

count = 0
for var in yugioh_data:
    sb.boxplot(data = yugioh_data[var], orient = "h", ax = axes[count,0])
    sb.histplot(data = yugioh_data[var], ax = axes[count,1])
    sb.violinplot(data = yugioh_data[var], orient = "h", ax = axes[count,2])
    count += 1

### Correlation Matrix of Numerical Variables

In [ ]:
# Correlation Matrix
print(yugioh_data.corr())

# Heatmap of the Correlation Matrix
f = plt.figure(figsize=(8, 8))
sb.heatmap(yugioh_data.corr(), vmin = -1, vmax = 1, linewidths = 1,
           annot = True, fmt = ".2f", annot_kws = {"size": 18}, cmap = "RdBu")

# Linear Regresstion of Numerical Values

In [ ]:
# Import essential models and functions from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Extract Response and Predictors
predictors = ["atk", "def", "level","scale","linkval"]
reponse_variable = ["card_prices"]

y = pd.DataFrame(yugioh_data[reponse_variable])
X = pd.DataFrame(yugioh_data[predictors])

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# Linear Regression using Train Data
linreg = LinearRegression()         # create the linear regression object
linreg.fit(X_train, y_train)        # train the linear regression model

# Coefficients of the Linear Regression line
print('Intercept of Regression \t: b = ', linreg.intercept_)
print('Coefficients of Regression \t: a = ', linreg.coef_)
print()

# Print the Coefficients against Predictors
print(pd.DataFrame(list(zip(X_train.columns, linreg.coef_[0])), columns = ["Predictors", "Coefficients"]))
print()

# Predict Response corresponding to Predictors
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

# Plot the training data and regression line.
f = plt.figure(figsize=(16, 8))
plt.scatter(X_train, y_train, color = "green")
#plt.scatter(X_test, y_test_pred, color = "red")
plt.axline((0, linreg.intercept_.item()), slope=linreg.coef_.item(), label='by slope', linewidth=4, color='r')
plt.show()

# Plot the test data and regression line.
f = plt.figure(figsize=(16, 8))
plt.scatter(X_test, y_test, color = "green")
#plt.scatter(X_test, y_test_pred, color = "red")
plt.axline((0, linreg.intercept_.item()), slope=linreg.coef_.item(), label='by slope', linewidth=4, color='r')
plt.show()

### Conclusion of linear regression


In [ ]:
# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train, y_train_pred))
print()

# Check the Prediction Accuracy (on Test Data)
print("Prediction Accuracy      \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print()

# EDA OF CATEGORICAL VALUES